In [14]:
from qiskit import *
from qiskit.assembler.run_config import RunConfig
from qiskit.compiler import transpile

In [15]:
qr = QuantumRegister(2)
cr = ClassicalRegister(2)
circuit = QuantumCircuit(qr, cr)
circuit.cz(qr[0],qr[1])
circuit.measure(qr, cr)
circuit.draw()

┌─┐   
q2_0: ─■─┤M├───
       │ └╥┘┌─┐
q2_1: ─■──╫─┤M├
          ║ └╥┘
c2_0: ════╩══╬═
             ║ 
c2_1: ═══════╩═

In [3]:
backend = Aer.get_backend('qasm_simulator')

In [4]:
circuit_out = transpile(circuit, backend)
circuit_out.draw()

┌─┐   
q0_0: ─■─┤M├───
       │ └╥┘┌─┐
q0_1: ─■──╫─┤M├
          ║ └╥┘
c0_0: ════╩══╬═
             ║ 
c0_1: ═══════╩═

In [5]:
from qiskit.assembler import assemble_circuits
from qiskit.qobj import QobjHeader
run_config= RunConfig(**{ 'shots': 1024})
qobj = assemble_circuits([circuit_out], qobj_id='jay', qobj_header=QobjHeader(description="testing header"), run_config=run_config)

In [6]:
qobj

QasmQobj(qobj_id='jay', config=QasmQobjConfig(memory_slots=2, n_qubits=2, shots=1024), experiments=[QasmQobjExperiment(config=QasmQobjExperimentConfig(memory_slots=2, n_qubits=2), header=QobjExperimentHeader(clbit_labels=[['c0', 0], ['c0', 1]], creg_sizes=[['c0', 2]], memory_slots=2, n_qubits=2, name='circuit0', qreg_sizes=[['q0', 2]], qubit_labels=[['q0', 0], ['q0', 1]]), instructions=[QasmQobjInstruction(name='cz', qubits=[0, 1]), QasmQobjInstruction(name='measure', qubits=[0], memory=[0]), QasmQobjInstruction(name='measure', qubits=[1], memory=[1])])], header=QobjHeader(description='testing header'))

In [7]:
def QobjPrint(qobj):
    print("header: "+ str(qobj.header))
    print("global user_config: " +  str(qobj.config))
    print("qobj_id: " + qobj.qobj_id)
    print("schema_version: " + qobj.schema_version)
    print("qobj_type: " + qobj.type)
    print("# experiments in qobj: " + str(len(qobj.experiments)))
    for experiment in qobj.experiments:
        print("    experiment header: "+ str(experiment.header))
        print("    experiment user_config: " + str(experiment.config))
        for instructions in experiment.instructions:
            print("        " + str(instructions))

In [8]:
QobjPrint(qobj)

header: QobjHeader(description='testing header')
global user_config: QasmQobjConfig(memory_slots=2, n_qubits=2, shots=1024)
qobj_id: jay
schema_version: 1.1.0
qobj_type: QASM
# experiments in qobj: 1
    experiment header: QobjExperimentHeader(clbit_labels=[['c0', 0], ['c0', 1]], creg_sizes=[['c0', 2]], memory_slots=2, n_qubits=2, name='circuit0', qreg_sizes=[['q0', 2]], qubit_labels=[['q0', 0], ['q0', 1]])
    experiment user_config: QasmQobjExperimentConfig(memory_slots=2, n_qubits=2)
        Instruction: cz
		qubits: [0, 1]

        Instruction: measure
		qubits: [0]
		memory: [0]

        Instruction: measure
		qubits: [1]
		memory: [1]



In [9]:
job = backend.run(qobj)
result = job.result()
print(result)

Result(backend_name='qasm_simulator', backend_version='0.5.0', date=datetime.datetime(2020, 4, 13, 20, 26, 42, 297006), header=Obj(description='testing header'), job_id='a75153ca-b9fa-4b14-8703-832c8052e3df', metadata={'max_memory_mb': 8192, 'omp_enabled': True, 'parallel_experiments': 1, 'time_taken': 0.005198339000000001}, qobj_id='jay', results=[ExperimentResult(data=ExperimentResultData(counts=Obj(0x0=1024)), header=Obj(clbit_labels=[['c0', 0], ['c0', 1]], creg_sizes=[['c0', 2]], memory_slots=2, n_qubits=2, name='circuit0', qreg_sizes=[['q0', 2]], qubit_labels=[['q0', 0], ['q0', 1]]), meas_level=<MeasLevel.CLASSIFIED: 2>, metadata={'measure_sampling': True, 'method': 'stabilizer', 'parallel_shots': 1, 'parallel_state_update': 8}, seed_simulator=630713068, shots=1024, status='DONE', success=True, time_taken=0.003227943)], status='COMPLETED', success=True, time_taken=0.006557941436767578)


In [10]:
result.get_counts()

{'00': 1024}